In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
        '../data/archive/train/mouth',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

validation_generator = train_datagen.flow_from_directory(
        '../data/archive/val/mouth',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 1069 images belonging to 2 classes.
Found 379 images belonging to 2 classes.


In [5]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 64)       0

In [8]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
      train_generator,
      validation_data = validation_generator,
      epochs=50,
      verbose=1,
      callbacks=early_stop)

Epoch 1/50
67/67 [==============================] - 28s 332ms/step - loss: 0.6614 - accuracy: 0.5622 - val_loss: 0.6062 - val_accuracy: 0.5778
Epoch 2/50
67/67 [==============================] - 8s 122ms/step - loss: 0.5740 - accuracy: 0.6679 - val_loss: 0.5783 - val_accuracy: 0.5778
Epoch 3/50
67/67 [==============================] - 8s 120ms/step - loss: 0.4984 - accuracy: 0.7530 - val_loss: 0.5829 - val_accuracy: 0.7045
Epoch 4/50
67/67 [==============================] - 8s 119ms/step - loss: 0.4673 - accuracy: 0.7736 - val_loss: 0.6239 - val_accuracy: 0.6095
Epoch 5/50
67/67 [==============================] - 8s 120ms/step - loss: 0.4297 - accuracy: 0.7998 - val_loss: 0.5950 - val_accuracy: 0.6332
Epoch 6/50
67/67 [==============================] - 8s 122ms/step - loss: 0.3781 - accuracy: 0.8148 - val_loss: 0.6341 - val_accuracy: 0.6807
Epoch 7/50
67/67 [==============================] - 9s 128ms/step - loss: 0.3075 - accuracy: 0.8644 - val_loss: 0.7747 - val_accuracy: 0.6332


In [9]:
test_generator = train_datagen.flow_from_directory(
        '../data/test/mouth',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        # Since we use binary_crossentropy loss, we need binary labels
        shuffle=True,
        class_mode='binary',
        batch_size=16)

Found 62 images belonging to 2 classes.


In [10]:
model.evaluate(test_generator)

4/4 [==============================] - 5s 1s/step - loss: 0.6785 - accuracy: 0.6129


[0.6784728169441223, 0.6129032373428345]

In [11]:
model.save("base_yawning_model.h5")

In [14]:
# predicting images
for im in os.listdir("../data/test/mouth"):
    path = "../data/test/mouth/" + im
    img = image.load_img(path, target_size=(224, 224, 3))
    x = image.img_to_array(img)
    x = x/255.0
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images)
    print(classes)
    if classes[0]>0.5:
        print(path+ ": is yawning")
    else:
        print(path+ " is not yawning")

PermissionError: [Errno 13] Permission denied: '../data/test/mouth/Normal'